# A Recommendation System for Restaurant Finder

In [1]:
# importing libraries
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import json 
from pandas.io.json import json_normalize 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
! pip install folium
import geopy.geocoders 
import folium 


Postal Codes in Toronto

In [2]:
df_toronto = pd.read_csv('toronto_base.csv') # created in Segmenting and Clustering Neighborhoods notebook
df_toronto.head(10)

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,7,M3B,North York,Don Mills,43.745906,-79.352188
8,8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937



### Create a Map of Toronto with its Postal Codes

In [3]:
df1 = df_toronto.loc[df_toronto['Postal Code'] == 'M9V']
neighborhood_latitude = df1['Latitude']
neighborhood_longitude = df1['Longitude']
print(neighborhood_latitude,neighborhood_longitude)

df_toronto.rename(columns={'Borough\n': 'Borough'}, inplace=True)
df_toronto.rename(columns={'Neighborhood\n': 'Neighborhood'}, inplace=True)

89    43.739416
Name: Latitude, dtype: float64 89   -79.588437
Name: Longitude, dtype: float64


In [4]:
# for the city Toronto, latitude and longtitude are extracted using M9V

my_map = folium.Map(location = [neighborhood_latitude, neighborhood_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    

my_map

### Focusing on the "Scarorough" Borough in Toronto (its neighborhoods)

In [5]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True).drop(columns = 'Unnamed: 0')
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Display a Map of Scarborough and Its Neighbourhood

In [6]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

In [7]:
def crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result1 = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result1.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result1;

In [8]:
CLIENT_ID = '0UOOVVQUXZTSOJ5302UDTB1L500GIY0RCTXSTUJYXUIFGJXG' # Hide Foursquare ID
CLIENT_SECRET = 'IBQQ0QYDBQCBCDZ3QBRSVC1JWKU0HGMIIMDVM4DOJW34RGAB' # Hide Foursquare Secret
VERSION = '20180605' # Foursquare API version

Using Foursquare database for Venues in the Neighborhoods inside "Scarborough"

In [9]:
Scarborough_foursquare_dataset =crawler(list(scarborough_data['Postal Code']),
                                        list(scarborough_data['Neighborhood']),
                                        list(scarborough_data['Latitude']),
                                        list(scarborough_data['Longitude']))


1.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Malvern, Rouge) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Rouge Hill, Port Union, Highland Creek) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1E (and Neighborhoods Guildwood, Morningside, West Hill) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1G (and Neighborhoods Woburn) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1K (and Neighborhoods Kennedy Park, Ionview, East Birchmount Park) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1L (and Neighborhoods Golden Mile, Clairlea, Oakridge) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffside, Cliffcrest, Scarborough Village West) SUCCESSFULLY.
10.
Data is Obtained, for the Postal Cod

### Preprocessing Data Received from Foursquare

In [10]:
def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [11]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venuse in Coordination "M1B" Posal Code and "Malvern, Rouge" Negihborhood(s) is:
20
Number of Venuse in Coordination "M1C" Posal Code and "Rouge Hill, Port Union, Highland Creek" Negihborhood(s) is:
5
Number of Venuse in Coordination "M1E" Posal Code and "Guildwood, Morningside, West Hill" Negihborhood(s) is:
26
Number of Venuse in Coordination "M1G" Posal Code and "Woburn" Negihborhood(s) is:
9
Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is:
33
Number of Venuse in Coordination "M1J" Posal Code and "Scarborough Village" Negihborhood(s) is:
13
Number of Venuse in Coordination "M1K" Posal Code and "Kennedy Park, Ionview, East Birchmount Park" Negihborhood(s) is:
27
Number of Venuse in Coordination "M1L" Posal Code and "Golden Mile, Clairlea, Oakridge" Negihborhood(s) is:
29
Number of Venuse in Coordination "M1M" Posal Code and "Cliffside, Cliffcrest, Scarborough Village West" Negihborhood(s) is:
13
Number of Venuse in Coordination "M1N" Pos

### Dispalying Venues for Each Neighborhood in Scarborough

In [12]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1B,"Malvern, Rouge",43.806686,-79.194353,Harvey's,This spot is popular,Restaurant,807
1,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,600
2,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,This spot is popular,Fast Food Restaurant,387
3,M1B,"Malvern, Rouge",43.806686,-79.194353,RBC Royal Bank,This spot is popular,Bank,906
4,M1B,"Malvern, Rouge",43.806686,-79.194353,Caribbean Wave,This spot is popular,Caribbean Restaurant,912


### Saving a Preprocessed of DataFrame as the Results from Foursquare

In [13]:
scarborough_venues.to_csv('scarborough_venues.csv')

### Summary about Neighborhoods inside "Scarborough"

In [14]:
Neighborhood_list = list(scarborough_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Scarborough:')
print(len(Neighborhood_list))
print('List of Neighborhoods inside Scarborough:')
Neighborhood_list

Number of Neighborhoods inside Scarborough:
16
List of Neighborhoods inside Scarborough:


['Malvern, Rouge',
 'Rouge Hill, Port Union, Highland Creek',
 'Guildwood, Morningside, West Hill',
 'Woburn',
 'Cedarbrae',
 'Scarborough Village',
 'Kennedy Park, Ionview, East Birchmount Park',
 'Golden Mile, Clairlea, Oakridge',
 'Cliffside, Cliffcrest, Scarborough Village West',
 'Birch Cliff, Cliffside West',
 'Dorset Park, Wexford Heights, Scarborough Town Centre',
 'Wexford, Maryvale',
 'Agincourt',
 "Clarks Corners, Tam O'Shanter, Sullivan",
 "Milliken, Agincourt North, Steeles East, L'Amoreaux East",
 "Steeles West, L'Amoreaux West"]

In [15]:
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,42,42,42,42,42,42,42
"Birch Cliff, Cliffside West",13,13,13,13,13,13,13
Cedarbrae,33,33,33,33,33,33,33
"Clarks Corners, Tam O'Shanter, Sullivan",33,33,33,33,33,33,33
"Cliffside, Cliffcrest, Scarborough Village West",13,13,13,13,13,13,13


### One-hot "categroies" Column into Unique Categorical Feature.

In [16]:
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Badminton Court,Bakery,Bank,Beach,Beer Store,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Hardware Store,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Pub,Rental Car Location,Rental Service,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,M1B,"Malvern, Rouge",43.806686,-79.194353,Harvey's,This spot is popular,807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy's,This spot is popular,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,This spot is popular,387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1B,"Malvern, Rouge",43.806686,-79.194353,RBC Royal Bank,This spot is popular,906,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M1B,"Malvern, Rouge",43.806686,-79.194353,Caribbean Wave,This spot is popular,912,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Defining Related Features for the Prediction for restaurant

In [17]:
list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'Asian Restaurant',
 'Bakery',
 'Breakfast Spot',
 'Burger Joint',
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Diner',

 'Fast Food Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint', 
 'Greek Restaurant',
 'Hakka Restaurant',
 
 'Hong Kong Restaurant',

 'Hotpot Restaurant',
 
 'Indian Restaurant',

 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',



 'Malay Restaurant',
 
 'Mediterranean Restaurant',
 
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 
 'Noodle House',
 
 'Pizza Place',
 
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 
 'Thai Restaurant',
 
 'Vegetarian / Vegan Restaurant',
 
 'Vietnamese Restaurant',
 'Wings Joint']

In [18]:
scarborough_onehot = scarborough_onehot[list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


scarborough_onehot.head()

,Asian Restaurant,Bakery,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Greek Restaurant,Hakka Restaurant,Hong Kong Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,1,2,1,0,0,1,2,6,0,0,0,0,0,0,0,1,0,1,0,1,0,1,1,1,0,0,1,1,1,2,1,1,0,0,0,1,0
"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
Cedarbrae,0,4,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
"Clarks Corners, Tam O'Shanter, Sullivan",0,0,0,0,0,1,1,1,0,2,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,2,0,2,1,0,1,1,0,1,0
"Cliffside, Cliffcrest, Scarborough Village West",0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0


### using k-means to Cluster Neighborhoods into 5 Clusters

In [19]:
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)

In [20]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Asian Restaurant,Bakery,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Greek Restaurant,Hakka Restaurant,Hong Kong Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Total Sum
G5,1.000000,2.000000,1.000000,0.000000,0.00,1.00,2.000000,6.000000,0.0,0.000000,0.00,0.00,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.0,1.000000,1.0,0.000000,0.0,1.000000,1.000000,1.000000,2.000000,1.0,1.0,0.00,0.000000,0.000000,1.0,0.0,27.000000
G4,1.500000,3.000000,0.000000,1.000000,0.00,0.00,0.500000,1.500000,0.0,1.500000,0.00,0.00,1.0,0.000000,0.5,0.0,0.000000,2.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000,1.500000,0.500000,0.0,0.0,0.00,0.500000,0.000000,0.5,0.5,17.500000
G2,0.000000,0.250000,0.250000,0.750000,0.25,0.25,0.250000,0.250000,0.0,1.000000,0.25,0.25,0.5,0.500000,0.0,0.0,0.000000,0.250000,0.250000,0.000000,0.250000,0.0,0.000000,0.0,0.250000,1.0,0.250000,3.250000,0.500000,0.750000,0.5,0.0,0.25,0.250000,0.000000,0.5,0.0,13.000000
G1,0.333333,1.000000,0.333333,0.333333,0.00,0.00,0.333333,4.000000,0.0,1.333333,0.00,0.00,0.0,0.000000,0.0,0.0,0.333333,0.333333,0.000000,0.000000,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.333333,2.000000,0.000000,0.666667,0.0,0.0,0.00,0.000000,0.333333,0.0,0.0,12.333333
G3,0.000000,0.333333,0.166667,0.166667,0.00,0.00,0.166667,0.333333,0.5,0.833333,0.00,0.00,0.0,0.166667,0.0,0.0,0.000000,0.166667,0.166667,0.166667,0.000000,0.0,0.000000,0.0,0.166667,0.0,0.000000,0.333333,0.833333,0.500000,0.0,0.0,0.00,0.166667,0.000000,0.0,0.0,5.166667


### Results

Best Group is G5;

Second Best Group is G4;

Third Best Group is G2;